In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import sys
from tqdm import trange
%load_ext line_profiler

from ipywidgets import interact, interactive, fixed, interact_manual
from matplotlib.widgets import Slider, Button

In [2]:
def plot_cubes(matrix):
    ax = plt.figure().add_subplot(projection='3d')
    ax.voxels(matrix,edgecolor='k')

In [3]:
def add_cube(matrix):
    pad_matrix = np.pad(matrix,1)
    conv = np.array([[[0,0,0],[0,1,0],[0,0,0]]
                     ,[[0,1,0],[1,-10,1],[0,1,0]]
                     ,[[0,0,0],[0,1,0],[0,0,0]]])

    places = scipy.ndimage.convolve(pad_matrix,conv,mode='constant',cval=0)
    
    new_matrices = []
    for idx in np.argwhere(places>0):
        new_mat = pad_matrix.copy()
        new_mat[tuple(idx)] = 1
        new_matrices.append(new_mat)
    return new_matrices

def add_cube_boolean(matrix):

    # Pad the boolean array
    pad_matrix = np.pad(matrix,1)

    # Define the convolution kernel
    conv = np.array([[[0,0,0],[0,1,0],[0,0,0]],
                     [[0,1,0],[1,0,1],[0,1,0]],
                     [[0,0,0],[0,1,0],[0,0,0]]])

    # Perform the convolution operation
    places = scipy.ndimage.convolve(pad_matrix.astype(int), conv, mode='constant', cval=0) > 0

    # Get the coordinates where a new cube can be added
    coords = np.argwhere(places & ~pad_matrix)

    new_matrices = []
    for coord in coords:
        new_matrix = pad_matrix.copy()
        new_matrix[tuple(coord)] = True
        new_matrices.append(new_matrix)
    
    return new_matrices

def trim_matrix(matrix):
    where = np.argwhere(matrix)
    min_coords = where.min(axis=0)
    max_coords = where.max(axis=0) + 1  # add 1 because the end index is exclusive
    matrix = matrix[min_coords[0]:max_coords[0], min_coords[1]:max_coords[1], min_coords[2]:max_coords[2]]
    return matrix

In [4]:
def get_distances(matrix,n:int=None):
    where = np.argwhere(matrix)
    dist_mat = scipy.spatial.distance_matrix(where,where)
    if not n:
        n = dist_mat.shape[0]
    distances = np.sort(dist_mat[np.triu_indices(n,k=1)])[::-1]
    d = distances
    return d

In [96]:
def check_chiral(matrix):
    reflection = matrix[::-1].copy()
    return not any([np.array_equal(matrix,_) for _ in rotations24(reflection)])

def rotations24(matrix):
    """List all 24 rotations of the given 3d array"""
    def rotations4(matrix, axes):
        """List the four rotations of the given 3d array in the plane spanned by the given axes."""
        for i in range(4):
             yield np.rot90(matrix, i, axes)

    # imagine shape is pointing in axis 0 (up)

    # 4 rotations about axis 0
    yield from rotations4(matrix, (1,2))

    # rotate 180 about axis 1, now shape is pointing down in axis 0
    # 4 rotations about axis 0
    yield from rotations4(np.rot90(matrix, 2, axes=(0,2)), (1,2))

    # rotate 90 or 270 about axis 1, now shape is pointing in axis 2
    # 8 rotations about axis 2
    yield from rotations4(np.rot90(matrix, axes=(0,2)), (0,1))
    yield from rotations4(np.rot90(matrix, -1, axes=(0,2)), (0,1))

    # rotate about axis 2, now shape is pointing in axis 1
    # 8 rotations about axis 1
    yield from rotations4(np.rot90(matrix, axes=(0,1)), (0,2))
    yield from rotations4(np.rot90(matrix, -1, axes=(0,1)), (0,2))

In [91]:
def add_n(df):
    nmax = df.n.max()
    new_df = pd.DataFrame(
            columns=[
                'id',
                'n',
                'matrix',
                'distances',
                'chiral',
                'id_from'
            ],
        )

    # Add a cube to each matrix
    new_df['matrix'] = df.loc[df.n==nmax].matrix.map(add_cube_boolean)
    new_df['id_from'] = df.loc[df.n==nmax]['id']
    
    # Explode the list of matrices
    new_df = new_df.explode('matrix')
    # Add the id of the matrix that it was created from
    new_df['matrix'] = new_df['matrix'].map(trim_matrix)
    new_df['id_from'] = new_df['id_from'].map(lambda x : list([x]))

    #Make the hash using the distance matrix
    new_df['distances'] = new_df.matrix.map(get_distances)
    new_df['id'] = new_df.distances.map(lambda x: hash(x.data.tobytes()))

    # Remove duplicates
    new_df2 = new_df.groupby(['id']).first()
    new_df2['id_from'] = new_df.groupby(['id']).id_from.sum()

    new_df2.reset_index(inplace=True)
    new_df2['n'] = nmax+1
    # Remove duplicates from the id_from column
    new_df2.id_from = new_df2.id_from.map(lambda x: list(set(x)))
    # Check chirality
    new_df2['chiral'] = new_df2.matrix.map(check_chiral) 
    
    df = pd.concat([df,new_df2],axis=0,ignore_index=True)
    return df

In [92]:
df0 = pd.DataFrame(
    columns=[
        'id',
        'n',
        'matrix',
        'distances',
        'chiral',
        'id_from'
    ],
    )
df0.loc[len(df0)] =  {
    
    'id':  hash(str(np.array([0]))),
    'n': 1,
    'matrix': np.array(
                        [[[1]]]
                        ,dtype='bool'
                        ),
    'distances': np.array([0]) ,
    'chiral': False,
    'id_from' : []
}

In [93]:
def add_till_n(n,df = df0):
    last_n = df.n.max()
    for i in trange(n-last_n):
        df = add_n(df)
    return df

In [105]:
df = add_till_n(11,df)

100%|██████████| 2/2 [10:08<00:00, 304.07s/it]


In [118]:
df['nums'] = df.chiral.apply(lambda x : 2 if x else 1)
total_nums = df.groupby('n').nums.sum()
total_nums

n
1           1
2           1
3           2
4           8
5          27
6         146
7         782
8        5308
9       35897
10     272726
11    2051415
Name: nums, dtype: int64

In [106]:
n = 11
ids = df.loc[df.n==n].id.tolist()
vals = df.loc[df.n==n].matrix.tolist()

@interact(x=(0,len(vals)-1))
def plot(x):
    print(ids[x])
    plot_cubes(vals[x])

interactive(children=(IntSlider(value=521338, description='x', max=1042676), Output()), _dom_classes=('widget-…